### Methods in this script parse COCO annotations, and corresponding .mp4 files, then convert them into a data format for YOLO training.

In [2]:
from google.colab import drive
# drive.mount("/content/drive") 

In [3]:
import json
import cv2
import os, glob

In [4]:
coco_root = glob.glob(os.getcwd() + "/**/" + "stroma nutsbolts/data", recursive = True)[0]
yolo_root = glob.glob(os.getcwd() + "/**/" + "stroma nutsbolts/yolo_data", recursive = True)[0]

In [ ]:
# converts bounding boxes in coco format to yolo format
def get_yolo_box(bbox, img_width=640, img_height=640):
  x, y, w, h = bbox[0], bbox[1], bbox[2], bbox[3]

  x_mid = ((x + (x+w)) /2) / img_width
  y_mid = ((y + (y+h)) /2) / img_height
  w = w / img_width
  h = h / img_height

  return format(x_mid, '.4f'), format(y_mid, '.4f'), format(w, '.4f'), format(h, '.4f')

In [ ]:
frames_to_get = {}

for subset in os.listdir(os.path.join(coco_root, "images")):  # subset stands for subsection of dataset, train, val, test

  # obtaining annotations for the subset
  json_file = open(coco_root + "/annotations/" + "instances_" + subset + ".json")
  annotations = json.load(json_file)['annotations']
  json_file.close()

  output_folder = os.path.join(yolo_root, subset)
  if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    os.mkdir(os.path.join(output_folder, "annotations"))
    os.mkdir(os.path.join(output_folder, "images"))
  else: # to remove previous annotation .txts
    annotation_files = glob.glob(os.path.join(output_folder, "annotations/*"))
    for file in annotation_files: 
      os.remove(file)

  frame_nums = []

  for annotation in annotations:
    frame = annotation['image_id']
    if frame not in frame_nums:
      frame_nums.append(frame)
    category = annotation['category_id']
    bbox = annotation['bbox']
    x, y, w, h = get_yolo_box(bbox)

    with open("{}/{:04d}.txt".format(output_folder + "/labels", frame), 'a') as f:
      f.write(f"{category-1} {x} {y} {w} {h}\n")
      f.close()
  
  frames_to_get[subset] = frame_nums

In [ ]:
# saves given frames as .jpg, i.e. the frames that have corresponding yolo annotations
def save_frames(video_file, frame_nums, images_folder):
  cap = cv2.VideoCapture(video_file)
  
  if not cap.isOpened(): print("!! Error on opening video file !!")

  frame_num = 0 
  while cap.isOpened():
    ret, frame = cap.read()

    if ret and (frame_num in frame_nums):
      cv2.imwrite("{}/{:04d}.jpg".format(images_folder, frame_num), frame)

    frame_num += 1

  cap.release()

In [ ]:
for subset, frames in frames_to_get.items():
  video_file = os.path.join(coco_root, "images", subset, subset + ".mp4")
  images_folder = os.path.join(yolo_root, subset, "images")

  save_frames(video_file, frames, images_folder)